<h1> 0. 모델 캐시 디렉토리 설정

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 먼저 불러오면서 캐시 디렉터리 설정
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it", 
    torch_dtype=torch.bfloat16, 
    cache_dir="/workspace/youngwoo/KOBCLLM/cache"
)

tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-2-9b-it", 
    cache_dir="/workspace/youngwoo/KOBCLLM/cache"
)

# 파이프라인 설정
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device="cuda",  # replace with "mps" to run on a Mac device
)

messages = [
    {"role": "user", "content": "Who are you? Please, answer in pirate-speak."},
]

outputs = pipe(messages, max_new_tokens=256)
assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)

<h1> 1. 데이터 전처리

In [ ]:
import pandas as pd

file_path = '/workspace/youngwoo/KOBCLLM/resource/신조선가/BULKER.xls' 
bulker_df = pd.read_excel(file_path, skiprows=1, engine='xlrd')
bulker_df = bulker_df.drop(columns=['번호'])
bulker_df = bulker_df.set_index('DATE')

print(bulker_df)

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

SYSTEM_PROMPT = """You're a helpful assistant"""

analysis_prompt = """당신은 데이터 분석 전문가입니다.

아래의 데이터 프레임은 BULKER선의 {COLUMNS}들의 신조선가 데이터 입니다.
{DATAFRAME}

사용자 요청에 대해 해당 데이터를 활용해 답변하세요.

# 사용자 요청
{MESSAGE}

답변은 분석한 결과 이외에 하지 않습니다."""

In [ ]:

def prompt_analysis_test(STATE):
    prompt = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(
            content=analysis_prompt.format_map(STATE)
        ),
    ]
    return prompt

# 모델과 토크나이저를 사용한 파이프라인 생성
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it", 
    torch_dtype=torch.bfloat16, 
    cache_dir="/workspace/youngwoo/KOBCLLM/cache"
)

tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-2-9b-it", 
    cache_dir="/workspace/youngwoo/KOBCLLM/cache"
)

gemma_llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device="cuda"  # 필요한 경우, 'mps'로 변경
)

In [11]:
# 데이터 준비 및 체인 실행
MESSAGE = "2년간 벌크선의 신조선가 추이에 대해 설명해주세요."


In [14]:
def prompt_analysis_test(STATE):
    prompt = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(
            content=analysis_prompt.format_map(STATE)
        ),
    ]
    return prompt

Chain__analysis_test = prompt_analysis_test | gemma_llm

TypeError: unsupported operand type(s) for |: 'function' and 'TextGenerationPipeline'

In [13]:
ANSWER = ""
for chunk in Chain__analysis_test.stream({"MESSAGE":MESSAGE,
                                        #   "DATAFRAME":df.to_markdown(index=False),
                                          "DATAFRAME":bulker_df.to_html(index=False),
                                          "COLUMNS":bulker_df.columns.to_list()}):
    print("\033[38;5;12m" + chunk.content + "\033[0m", end="", flush=True)

NameError: name 'Chain__analysis_test' is not defined